# Retrieval Augmented Generation with LLMs
This notebook demonstrates the process of implementing retrieval-augmented generation (RAG) with
Langchain LLMs.

# The LLM Pipeline

In [1]:
import chromadb

client = chromadb.PersistentClient(path='../datasets/processed/chroma_db')
print(client.list_collections())

[Collection(name=summed), Collection(name=instruction), Collection(name=ingredient), Collection(name=name)]


## Loading the models and vector stores using Langchain

In [2]:
import torch
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.embeddings import HuggingFaceBgeEmbeddings

# Initialize models
model_name = "BAAI/bge-large-en"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

model_path = "TheBloke/Llama-2-13B-chat-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map="auto", revision="main"
)

# Initialize Chroma
chroma_store = Chroma(collection_name='summed', persist_directory='../datasets/processed/chroma_db', embedding_function=hf)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    # temperature=0.00,
    # top_p=0.95,
    repetition_penalty=1.15,
)
llm = HuggingFacePipeline(pipeline=pipe)

/home/alvin/miniconda3/envs/lc-update/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Relevant documents can be retrieved from the vector store and used as context

In [9]:
sample_recipe = chroma_store.similarity_search("Chicken Noodle Soup", k=5)
test_recipe = chroma_store.similarity_search('Seared Tuna',k=1)[0].page_content
for doc in sample_recipe:
    print(doc)

noodle_recipe = sample_recipe[0].page_content

page_content='Recipe Name: Chocolate-Glazed Pound Cake, \nIngredients: , \nCooking Instructions: 1. Microwave 6 ounces chopped bittersweet chocolate with 1 stick cut-up butter and 1 tablespoon honey on 75 percent power until melted, about 2 minutes. Whisk until smooth. Place a loaf of pound cake on a rack and drizzle with the glaze., '
page_content='Recipe Name: Instant Chocolate Cake, \nIngredients: , \nCooking Instructions: 1. Whisk 1/4 cup flour, 5 tablespoons sugar, 2 tablespoons cocoa powder, 1 egg, 3 tablespoons milk, 3 tablespoons vegetable oil and a dash of vanilla extract and salt in a large mug until smooth. Microwave until puffed, about 2 minutes.\n2. Copyright (c)2011 Food Network Magazine, LLC. Published by Hyperion. Available wherever books are sold. All Rights Reserved. Photograph courtesy of Antonis Achilleos., '
page_content='Recipe Name: Chocolate-Marshmallow Mug Cakes, \nIngredients: , \nCooking Instructions: 1. To make a batch, whisk 1/2 cup each sugar and buttermil

In [4]:
print(noodle_recipe)

Recipe Name: Chicken Noodle Soup, 
Ingredients: 3 liters (about 12 1/2 cups) Chicken Stock, recipe follows, 2 chicken breasts (about 450 grams/1 pound), 2 teaspoons olive oil, 1/4 red or yellow onion, diced, 2 carrots, peeled and cut into 1-centimeter (about 1/2-inch) rounds, 1 stalk celery, cut into 1-centimeter (about 1/2-inch) slices, 1/4 fennel bulb, diced, Kosher salt, Freshly ground black pepper, 1 garlic clove, minced, 1/4 teaspoon turmeric, 1 1/2 cups broad egg noodles, 2 small handfuls Swiss chard or kale, stems removed and roughly chopped, 1/2 lemon, juiced, 2 tablespoons finely chopped dill, parsley or fennel fronds, 1 1/4 to 1 1/2 kilograms whole chicken carcasses (2 3/4 to 3 1/3 pounds), 3 celery stalks, 3 medium carrots, 1 large yellow onion, 1/2 head of garlic, 1 handful parsley stalks, 5 sprigs thyme, 2 bay leaves, 1 teaspoon black peppercorns, 2 teaspoon kosher salt, 
Cooking Instructions: 1. Pour the chicken stock into a large saucepan and add in the chicken breasts. 

In [5]:
print(test_recipe)

Recipe Name: Seared Tuna, 
Ingredients: 2 pounds sushi-grade ahi tuna, cut into 1-inch cubes, 1/2 cup olive oil, Salt and freshly ground black pepper, Rosemary stems cut into skewers about 2-inches long with leaves attached, 
Cooking Instructions: 1. Preheat a grill or grill pan to high.
2. Put cubed tuna in a mixing bowl with olive oil, salt and pepper. Toss to incorporate the seasoning. Skewer the tuna through the center half way down the skewer. Season again with salt and pepper. Sear on a very hot grill for 10 seconds on all sides. Cool and serve at room temperature., 


In [8]:
irrelevant_recipe = chroma_store.similarity_search("Chocolate Cake", k=5)[0].page_content
print(irrelevant_recipe)

Recipe Name: Chocolate-Glazed Pound Cake, 
Ingredients: , 
Cooking Instructions: 1. Microwave 6 ounces chopped bittersweet chocolate with 1 stick cut-up butter and 1 tablespoon honey on 75 percent power until melted, about 2 minutes. Whisk until smooth. Place a loaf of pound cake on a rack and drizzle with the glaze., 


### LLM Performance without retrieval-augmentation

In [ ]:
#

### The LLM chain inserts these documents into the input prompt as additional context. 

In [7]:
template = f"""
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

### Instruction:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{test_recipe}

### Input:
How would you rewrite this recipe for chicken noodle soup in order to substitute chicken for seared tuna? Please provide your answer in the form of an informative cooking recipe.
{noodle_recipe}

### Response:
"""

test_result = llm(template)
print(test_result)

/home/alvin/miniconda3/envs/langchain/lib/python3.11/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


To substitute chicken for seared tuna in this recipe, you can follow these steps:

1. Replace the 1 (4 pound) whole chicken with 1 (4 pound) piece of seared tuna.
2. Keep all the vegetables and seasonings the same, except for the chicken broth, which will now be used as a marinade for the tuna.
3. Marinate the tuna in the chicken broth, along with the lemongrass, bay leaves, and peppercorns, for at least 1 hour in the refrigerator.
4. Before cooking the tuna, remove it from the marinade and pat it dry with paper towels to remove excess moisture.
5. In step 3 of the original recipe, use a skillet or cast-iron pan to sear the tuna over high heat for 30 seconds to 5 minutes per side, or until it reaches your desired level of doneness.
6. Once the tuna is cooked, add it to the pot with the vegetables and noodles, and simmer for an additional 10 minutes to allow the flavors to meld together.

By following these steps, you will have successfully substituted chicken for seared tuna in this re

#### The structure of the response can be manipulated by providing the LLM with the start of the expected answer. Here, this is achieved by appending "Recipe Name:" to the end of the input query. The LLM will use this template and the inserted recipes (which have a very similar form) to generate an output that looks like a recipe you might find on the internet.

In [8]:
template = f"""
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

### Instruction:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{test_recipe}

### Input:
How would you rewrite this recipe for chicken noodle soup in order to add seared tuna? Please provide your answer in the form of an informative cooking recipe.
{noodle_recipe}

### Response: Recipe Name:
"""

print(llm(template))

Asian Sesame Seared or Grilled Tuna (Gluten Free)

To incorporate seared tuna into the existing chicken noodle soup recipe, you can follow these steps:

Step 1: Prepare the tuna marinade by whisking together 0.25 cup tamari (gluten-free soy sauce), 0.25 cup sesame oil, 2 tablespoons mirin (Japanese sweet wine), 2 tablespoons honey, 2 tablespoons rice wine vinegar, 2 tablespoons grated fresh ginger, 2 green onions, thinly sliced, divided, 2 cloves garlic, minced, or more to taste, in a bowl until well combined.

Step 2: In a separate bowl, mix together 1 pound carrots, 0.66666668653488 bunch celery, trimmed, 1 large onion, peeled and halved, 3 tablespoons chopped lemongrass, 3 large bay leaves, 10 whole black peppercorns, and 1 pound carrots, peeled and sliced, in a large stockpot.

Step 3: Add 1 (4 pound) whole chicken, cut into pieces, to the stockpot and pour in 1 gallon water. Bring to a boil over high heat, then reduce the heat, cover, and simmer for 30 minutes, or until the chicke

#### What would happen if a recipe irrelevant to the input query was provided? The LLM will ignore the information in the recipe and fall back on the knowledge base it was trained on. Prompting the structure answer by appending "Recipe Name:" still achieves the desired effect.

In [9]:
template = f"""
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

### Instruction:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{irrelevant_recipe}

### Input:
How would you rewrite this recipe for chicken noodle soup in order to substitute chicken for seared tuna? Please provide your answer in the form of an informative cooking recipe.
{noodle_recipe}

### Response:
"""

print(llm(template))

To make this delicious chicken noodle soup with seared tuna instead of chicken, simply follow these steps:

Ingredients:

* 1 (4 pound) piece of sushi-grade tuna, cut into pieces
* 1 gallon water
* 1 pound whole carrots, peeled and sliced
* 0.66666668653488 bunch celery, trimmed
* 1 large onion, peeled and halved
* 3 tablespoons chopped lemongrass
* 3 large bay leaves
* 10 whole black peppercorns
* 1 pound carrots, peeled and sliced
* 0.33333334326744 bunch celery with leaves, sliced
* 0.25 cup tuna soup base (available at most Asian grocery stores)
* 1 (8 ounce) package dry egg noodles

Cooking Instructions:

1. Place tuna into a large stockpot and pour in water. Bring to a boil over high heat.
2. Reduce the heat, cover, and simmer for 30 minutes, or until the tuna is cooked through and flakes easily with a fork.
3. Add whole carrots, whole celery, onion half, lemongrass, bay leaves, and peppercorns. Cover and simmer for 1 hour; the tuna should be opaque and the juices should run clea

In [10]:
template = f"""
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

### Instruction:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{irrelevant_recipe}

### Input:
How would you rewrite this recipe for chicken noodle soup in order to add seared tuna? Please provide your answer in the form of an informative cooking recipe.
{noodle_recipe}

### Response:
Recipe Name:"""


print(llm(template))

 Tasty Tuna Chicken Noodle Soup, Ingredients: 1 (4 pound) whole chicken, cut into pieces, 1 gallon water, 1 pound whole carrots, 0.66666668653488 bunch celery, trimmed, 1 large onion, peeled and halved, 3 tablespoons chopped lemongrass, 3 large bay leaves, 10 whole black peppercorns, 1 pound carrots, peeled and sliced, 0.33333334326744 bunch celery with leaves, sliced, divided, 0.25 cup chicken soup base, 1 (8 ounce) package dry egg noodles, Cooking Instructions: Place chicken into a large stockpot and pour in water. Bring to a boil over high heat. Reduce the heat, cover, and simmer, skimming fat as needed, for 30 minutes. Add whole carrots, whole celery, onion half, lemongrass, bay leaves, and peppercorns. Cover and simmer for 1 hour; chicken should no longer be pink at the bone and the juices should run clear. An instant-read thermometer inserted into the thickest part of the thigh, near the bone, should read 165 degrees F (74 degrees C)., Transfer chicken to a platter and let sit un

#### The response from the first LLM query can be fed back into the LLM with a different template to create the final, modified recipe.

In [11]:
modify_template = f"""
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

### Instruction:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{noodle_recipe}

### Input:
Rewrite this recipe to include the modifications provided.
{test_result}

### Response: Recipe Name:
"""

print(llm(modify_template))

Awesome Tuna Noodle Soup

Ingredients:

* 1 (4 pound) piece of seared tuna, cut into bite-sized pieces
* 1 gallon water
* 1 pound whole carrots, peeled and sliced
* 0.66666668653488 bunch celery, trimmed, sliced (not the leaves)
* 1 large onion, peeled and halved
* 3 tablespoons chopped lemongrass
* 3 large bay leaves
* 10 whole black peppercorns
* 1 pound carrots, peeled and sliced
* 0.33333334326744 bunch celery with leaves, sliced, divided
* 0.25 cup tuna broth base
* 1 (8 ounce) package dry egg noodles

Cooking Instructions:

1. Place tuna into a large stockpot and pour in water. Bring to a boil over high heat.
2. Reduce the heat, cover, and simmer for 1 hour; tuna should be opaque and flake easily with a fork.
3. Remove tuna from the pot and let it rest for 10 minutes before cutting it into bite-sized pieces.
4. Strain the liquid from the pot and discard any solids.
5. Return the strained liquid to the pot and stir in sliced carrots, sliced celery, lemongrass, bay leaves, and pepp

### Intermediate chain-of-thought and few-shot examples

In [ ]:
fewshot_cot_template = f"""
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

### Instruction:
Use the following examples to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Input: How this recipe for chicken noodle soup be rewritten to use seared tuna instead of chicken?
{noodle_recipe}
Chicken needs to be substituted for tuna, so it should be removed from the ingredient list and replaced with an equal amount of tuna. Now that chicken is no longer in the ingredient list,
the recipe instructions that reference chicken need to be adjusted to include tuna instead.

### Input:
Rewrite this recipe to include the modifications provided.
{test_result}

### Response: Recipe Name:
"""

### Implementing the LLM chain in Langchain

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, StuffDocumentsChain, LLMChain
template_llm = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

### Instruction:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}

### Input:
{question}

### Response:
\n
"""
prompt = PromptTemplate(template=template_llm, input_variables=["context", "question"])
document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)

recipe_retriever = chroma_store.as_retriever(search_kwargs={"k": 1})

analyze_and_modify = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=recipe_retriever,
    chain_type_kwargs={
        "prompt": prompt,
        "document_variable_name": "context",
        "document_prompt": document_prompt,
    },
)